# A first look at the data

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

Open questions:
* How do I make sure that there are not two people who have the same name?
    * Partei, Beruf, Ort, Fraktion, etc sind alle nicht eindeutig. Geb_jahr könte helfen, aber ist nicht für alle vorhanden...

### 1. Simple metrics for the network

Load the data

In [2]:
file_abg = Path('../Daten_Thurgau/Mitglieder_GR_2008_2021.csv')
file_vor = Path('../Daten_Thurgau/Vorstoesse_GR_ab_2012_Themen_def.csv')

In [3]:
vor = pd.read_csv(file_vor,delimiter=';')
abg = pd.read_csv(file_abg,delimiter=';')

In [4]:
abg_2012 = abg.loc[abg['Jahr']>=2012]
abg_list = pd.unique(abg_2012['Vorname'] + ' ' + abg_2012['Name'])
print(f'Wir haben {len(abg_list)} unabhängige Abgeordnete seit 2012') 

Wir haben 275 unabhängige Abgeordnete seit 2012


In [8]:
vor

,DATUM_VORSTOSS_EINGANG,NR_LEGISLATUR,GRG_NR,VORSTOSS_BEZEICHNUNG,VORSTOSSART_BEZEICHNUNG,VORSTOSSART_CODE,Sachbegriffe_GRGEKO_1,Sachbegriffe_GRGEKO_2,Sachbegriffe_GRGEKO_3,Thema_1,...,VORSTOESSER_IN_10,PARTEI_BEZEICHNUNG_10,PARTEI_CODE_10,BEANTWORTUNG_REGIERUNGSRAT,DATUM_BESCHLUSS_GR,BESCHLUSS_GR,BESCHLUSS_GR_CODE,VORLAEUFIGE_UNTERSTUETZUNG,ERLEDIGUNG_GR,DATUM_BEANTWORTUNG_REGIERUNGSRAT
0,03.03.2010,0205/2008-2012/GR,205,Umsetzungstermin des Auszahlungsmodus der Präm...,Motion,2,"83.2 Kranken- und Unfallversicherung, Vereinba...",NaN,NaN,Sozialer Schutz,...,NaN,NaN,NaN,30.03.2010,05.05.2010,erheblich,1.0,NaN,NaN,NaN
1,27.10.2010,0293/2008-2012/GR,293,Änderung von § 9 des Gesetzes über das Stimm- ...,Motion,2,16.2 Abstimmungen,NaN,NaN,Staatspolitik,...,NaN,NaN,NaN,23.08.2011,26.10.2011,erheblich,1.0,NaN,NaN,NaN
2,27.10.2010,0294/2008-2012/GR,294,Abbau der Thurgauer Warteliste in der kostende...,Motion,2,"73.3 Andere Energien: Gas, Fernwärme, Solarene...",NaN,NaN,Energie,...,NaN,NaN,NaN,27.09.2011,25.01.2012,nicht erheblich,2.0,NaN,NaN,NaN
3,16.02.2011,0312/2008-2012/GR,312,Einführung Familienabzug im Steuergesetz,Motion,2,"64.1 Staats- und Gemeindesteuern, allgemeine O...",NaN,NaN,Steuern,...,NaN,NaN,NaN,20.09.2011,09.11.2011,erheblich,1.0,NaN,NaN,NaN
4,16.03.2011,0321/2008-2012/GR,321,Ergänzungsleistungen für einkommensschwache Fa...,Motion,2,"83.1 AHV, IV, EL",NaN,NaN,Sozialer Schutz,...,NaN,NaN,NaN,06.03.2012,28.03.2012,nicht erheblich,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,08.12.2021,0251/2020-2024/GR,251,Nur wenige Energiestädte aus dem Thurgau,Einfache Anfrage,5,"73.1 Allgemeines, Energiegesetz, Kernenergie, ...",NaN,NaN,Energie,...,NaN,NaN,NaN,pendent,NaN,NaN,NaN,NaN,NaN,NaN
997,08.12.2021,0254/2020-2024/GR,254,Verbesserungen für die Pflege sind dringend,Interpellation,4,NaN,NaN,NaN,Gesundheit,...,NaN,NaN,NaN,pendent,pendent,NaN,NaN,NaN,NaN,NaN
998,08.12.2021,0255/2020-2024/GR,255,Planungsstand Oberlandstrasse: Steckt die OLS ...,Interpellation,4,"72.3 Strassenwesen: Gesetz, Strassenbau, Autob...",NaN,NaN,Raumplanungs- und Wohnungswesen,...,NaN,NaN,NaN,pendent,pendent,NaN,NaN,NaN,NaN,NaN
999,08.12.2021,0253/2020-2024/GR,253,Faire Löhne für den Kanton Thurgau,Interpellation,4,NaN,NaN,NaN,Beschäftigung und Arbeit,...,NaN,NaN,NaN,pendent,pendent,NaN,NaN,NaN,NaN,NaN


In [23]:
vor_test

,DATUM_VORSTOSS_EINGANG,NR_LEGISLATUR,GRG_NR,VORSTOSS_BEZEICHNUNG,VORSTOSSART_BEZEICHNUNG,VORSTOSSART_CODE,Sachbegriffe_GRGEKO_1,Sachbegriffe_GRGEKO_2,Sachbegriffe_GRGEKO_3,Thema_1,...,VORSTOESSER_IN_10,PARTEI_BEZEICHNUNG_10,PARTEI_CODE_10,BEANTWORTUNG_REGIERUNGSRAT,DATUM_BESCHLUSS_GR,BESCHLUSS_GR,BESCHLUSS_GR_CODE,VORLAEUFIGE_UNTERSTUETZUNG,ERLEDIGUNG_GR,DATUM_BEANTWORTUNG_REGIERUNGSRAT
0,03.03.2010,0205/2008-2012/GR,205,Umsetzungstermin des Auszahlungsmodus der Präm...,Motion,2,"83.2 Kranken- und Unfallversicherung, Vereinba...",NaN,NaN,Sozialer Schutz,...,NaN,NaN,NaN,30.03.2010,05.05.2010,erheblich,1.0,NaN,NaN,NaN
1,27.10.2010,0293/2008-2012/GR,293,Änderung von § 9 des Gesetzes über das Stimm- ...,Motion,2,16.2 Abstimmungen,NaN,NaN,Staatspolitik,...,NaN,NaN,NaN,23.08.2011,26.10.2011,erheblich,1.0,NaN,NaN,NaN
2,27.10.2010,0294/2008-2012/GR,294,Abbau der Thurgauer Warteliste in der kostende...,Motion,2,"73.3 Andere Energien: Gas, Fernwärme, Solarene...",NaN,NaN,Energie,...,NaN,NaN,NaN,27.09.2011,25.01.2012,nicht erheblich,2.0,NaN,NaN,NaN
3,16.02.2011,0312/2008-2012/GR,312,Einführung Familienabzug im Steuergesetz,Motion,2,"64.1 Staats- und Gemeindesteuern, allgemeine O...",NaN,NaN,Steuern,...,NaN,NaN,NaN,20.09.2011,09.11.2011,erheblich,1.0,NaN,NaN,NaN
4,16.03.2011,0321/2008-2012/GR,321,Ergänzungsleistungen für einkommensschwache Fa...,Motion,2,"83.1 AHV, IV, EL",NaN,NaN,Sozialer Schutz,...,NaN,NaN,NaN,06.03.2012,28.03.2012,nicht erheblich,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,04.10.2021,0231/2020-2024/GR,231,Politische Rechte für Menschen mit Behinderung,Motion,2,NaN,NaN,NaN,Staatspolitik,...,NaN,NaN,NaN,pendent,NaN,pendent,5.0,NaN,NaN,NaN
981,04.10.2021,0230/2020-2024/GR,230,Finanzierung der familien- und schulergänzende...,Motion,2,NaN,NaN,NaN,Beschäftigung und Arbeit,...,NaN,NaN,NaN,pendent,NaN,pendent,5.0,NaN,NaN,NaN
987,27.10.2021,0238/2020-2024/GR,238,Strassenverkehrsabgaben - Weniger Gebühren wär...,Leistungsmotion,3,"74.1 Strassenverkehr, öffentlicher Verkehr auf...",NaN,NaN,Verkehr,...,NaN,NaN,NaN,pendent,NaN,pendent,5.0,NaN,NaN,NaN
994,24.11.2021,0247/2020-2024/GR,247,Einheitliche Steuersoftware für Kanton und Gem...,Motion,2,NaN,NaN,NaN,Staatspolitik,...,NaN,NaN,NaN,pendent,NaN,pendent,5.0,NaN,NaN,NaN


In [22]:
vor_test=vor[vor['VORSTOSSART_CODE'].isin([1,2,3,7])]

In [25]:
vor_abg = vor_test[[f'VORSTOESSER_IN_{i}' for i in range(1,11)]]
netzwerk = pd.DataFrame(columns=['Name','Colab','Vorst'])
for name in abg_list: # loop over the abgeordneten
    name_vorstoesse = [] # stores vorstoss indexes where this person was part of
    name_colab = [] # stores names of colaborators
    for i in vor_test.index: # loop over the vorstoesse
        unterz = list(vor_abg.loc[i].dropna())
        if name in unterz: # check if this person has supported the vorstoss
            name_vorstoesse.append(i)
            for u in unterz: # loop over unterzeichner
                if u not in name_colab and u != name: name_colab.append(u)
    netzwerk.loc[netzwerk.shape[0]] = [name,name_colab,name_vorstoesse]

In [112]:
netzwerk


,Name,Colab,Vorst
0,Inge Abegglen,"[Alex Frei, Hans Munz, Helen Jordi, Erwin Imhof]",[136]
1,Wolfgang Ackerknecht,"[Peter Bühler, Max Möckli, Daniel Vetterli, R...","[138, 273, 421, 428, 430, 535, 538, 606, 711]"
2,Elsbeth Aepli Stettler,"[Christian Koch, Marlies Näf, Walter Schönholz...","[39, 52]"
3,Margrit Aerne,"[Roland A. Huber, Cäcilia Bosshard, Turi Schal...","[116, 462]"
4,Clemens Albrecht,[],[]
...,...,...,...
270,David Zimmermann,"[Hans Munz, Hanspeter Gantenbein, Josef Gemper...","[127, 402, 424, 467, 478, 491, 658, 699, 958, ..."
271,Andreas Zuber,[Ruedi Zbinden],[983]
272,Käthi Zürcher,[],[]
273,Felix Züst,[],[]


Check what is the mean, median, min, max connectivity

In [114]:
print('Mean connectivity:', netzwerk['Colab'].apply(len).mean())
print('Median connectivity:', netzwerk['Colab'].apply(len).median())
print('Max connectivity:', netzwerk['Colab'].apply(len).max())

Mean connectivity: 12.04
Median connectivity: 6.0
Max connectivity: 71


In [26]:
print('Mean connectivity:', netzwerk['Colab'].apply(len).mean())
print('Median connectivity:', netzwerk['Colab'].apply(len).median())
print('Max connectivity:', netzwerk['Colab'].apply(len).max())

Mean connectivity: 6.807272727272728
Median connectivity: 3.0
Max connectivity: 43


In [27]:
netzwerk.loc[netzwerk['Colab'].apply(len).idxmax()]

Name                                        Josef Gemperle
Colab    [Toni Kappeler, Bernhard Wälti, Thomas Böhni, ...
Vorst    [5, 12, 25, 32, 74, 86, 87, 105, 213, 240, 403...
Name: 64, dtype: object

In [130]:
abg_2012.loc[(abg_2012['Vorname']=='Josef') & (abg_2012['Name']=='Gemperle')]

,Jahr,Name,Vorname,Beruf,Ort,Bezirk,Fraktion,GR_Eintritt,Geb_Jahr,Partei,Geschlecht
450,2013,Gemperle,Josef,Meisterlandwirt,Fischingen,Münchwilen,CVP/glp,2004,NaN,CVP,männlich
451,2014,Gemperle,Josef,Meisterlandwirt,Fischingen,Münchwilen,CVP/glp,2004,NaN,CVP,männlich
452,2015,Gemperle,Josef,Meisterlandwirt,Fischingen,Münchwilen,CVP/glp,2004,NaN,CVP,männlich
453,2016,Gemperle,Josef,Meisterlandwirt,Fischingen,Münchwilen,CVP/glp,2004,NaN,CVP,männlich
454,2017,Gemperle,Josef,Meisterlandwirt,Fischingen,Münchwilen,CVP/EVP,2004,NaN,CVP,männlich
455,2018,Gemperle,Josef,Meisterlandwirt,Fischingen,Münchwilen,CVP/EVP,2004,NaN,CVP,männlich
456,2019,Gemperle,Josef,Meisterlandwirt,Fischingen,Münchwilen,CVP/EVP,2004,NaN,CVP,männlich
457,2020,Gemperle,Josef,Meisterlandwirt,Fischingen,Münchwilen,CVP/EVP,2004,NaN,CVP,männlich
458,2021,Gemperle,Josef,Meisterlandwirt,Fischingen,Münchwilen,CVP/EVP,2004,NaN,CVP,männlich
462,2012,Gemperle,Josef,Meisterlandwirt,Fischingen,Münchwilen,CVP/GLP,2004,1960.0,CVP,männlich
